In [1]:
### --------------
##  Using several algor to do the kaggle Taitannike prediction
## Python3
## 2019-12-13

In [1]:
import numpy as np
import pandas as pd
import time

### 1 数据读取并填充空白数据

In [2]:
train = pd.read_csv("titanic_train.csv")
test  = pd.read_csv("test.csv")
full  = train.append(test,ignore_index = 'True')
print("训练数据大小:",train.shape)
print("预测数据大小:",test.shape)
print("全体数据大小:",full.shape)
full.head(3)

训练数据大小: (891, 12)
预测数据大小: (418, 11)
全体数据大小: (1309, 12)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282


In [3]:
train['Embarked']

0      S
1      C
2      S
3      S
4      S
5      Q
6      S
7      S
8      S
9      C
10     S
11     S
12     S
13     S
14     S
15     S
16     Q
17     S
18     S
19     C
20     S
21     S
22     Q
23     S
24     S
25     S
26     C
27     S
28     Q
29     S
      ..
861    S
862    S
863    S
864    S
865    S
866    C
867    S
868    S
869    S
870    S
871    S
872    S
873    S
874    C
875    C
876    S
877    S
878    S
879    C
880    S
881    S
882    S
883    S
884    S
885    Q
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, dtype: object

### 1.1 数据的一般特性

In [3]:
train.describe()

C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 1.2 数据的缺失数据

In [4]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


### 1.3 填充空白数据

In [5]:
### 鉴于 Embarked 缺失 891 - 889 = 2  
### 但是这一列数据是 字符串并不数字，并不能通过均值来填充
from collections import Counter 
Counter(full['Embarked'])  # 'S' 数量角度可以用来填充
full['Embarked'] =  full['Embarked'].fillna('S')

In [6]:
### 鉴于Age and Fare 的数据类型是数值型，可以通过平均值来填充
full['Age']  = full['Age'].fillna(full['Age'].mean())
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())

In [7]:
### 'Cabin ' - 客舱数据的缺失
full['Cabin'].head(5),#Counter(full['Cabin'])

(0     NaN
 1     C85
 2     NaN
 3    C123
 4     NaN
 Name: Cabin, dtype: object,)

In [8]:
## 由于 数据 缺失角度，所以填充未知 "UNK"
full['Cabin']  = full['Cabin'].fillna('UNK')

In [9]:
### 填充后数据已经全部相等
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


### 2 特征提取

### 2.1将性别'sex' 通过数字来表示

In [10]:
sex_map_dict = {'male':1,'female':0}
full['Sex']  = full['Sex'].map(sex_map_dict)

### 2.2 将特征'Embarked' 通过one - hot来表示 每一个旅客的特征都可以用one-hot来表示

In [11]:
### 2.2 将特征'Embarked' 通过one - hot来表示 每一个旅客的特征都可以用one-hot来表示
### 比如‘Embarked’ 有 ‘C’,‘Q’,'S'三个特征，每个旅客都只会在一个特征上有表示为'1'，其他表示为'0'
Counter(full['Embarked'])

Counter({'C': 270, 'Q': 123, 'S': 916})

In [12]:
Embarked_DF = pd.DataFrame() # 存放提取后的特征
Embarked_DF = pd.get_dummies(full['Embarked'],prefix = 'Embarked')
Embarked_DF.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


In [13]:
### 将参数的虚拟one_hot向量加入到full中
full = pd.concat([full,Embarked_DF],axis =1)  # axis = 1 按照列来添加
full.drop('Embarked',axis = 1,inplace = True)  # 去掉虚拟one-hot变量:Embarked_one_hot

In [14]:
full.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,Embarked_S
0,22.0,UNK,7.2500,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,0.0,0.0,1.0
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,1.0,0.0,0.0


### 2.3 将客舱等级Pclass 特征提取  - 转换成one-hot变量

In [15]:
### 2.3 将客舱等级Pclass 特征提取  - 转换成one-hot变量
Pclass_DF    = pd.DataFrame()
Pclass_DF    = pd.get_dummies(full['Pclass'],prefix = 'Pclass')
Pclass_DF.head()

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [16]:
# 将Pclass_one_hot 添加到full中,
full = pd.concat([full,Pclass_DF],axis = 1)
full.drop('Pclass',axis = 1,inplace = 'True')  #去掉虚拟变量
full.head(2)

,Age,Cabin,Fare,Name,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,UNK,7.2500,"Braund, Mr. Owen Harris",0,1,1,1,0.0,A/5 21171,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,C85,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,0,1,1.0,PC 17599,1.0,0.0,0.0,1.0,0.0,0.0


### 2.4 从旅客的名字中获得 有效的特征

In [17]:
### 2.4 从旅客的名字中获得 有效的特征
full['Name'].head(10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7                       Palsson, Master. Gosta Leonard
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [18]:
def get_title(Name):
    last_name   = Name.split(',')[1]
    title       = last_name.split('.')[0]
    Title       = title.strip()
    return Title
# 基于上述函数抽取出数据的title - 并新建成一列 Title_tmp['Title']
Title_DF = pd.DataFrame()
Title_DF['Title']  = full['Name'].map(get_title)
Counter(Title_DF['Title'])  # 名字中可能出现的种类
Title_DF.head()

,Title
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr


In [19]:
### 将title重新map成完整的社会称谓
map_dict = {
    'Capt'  :'Officer',
    'Col'   :'Officer',
    'Major' :'Officer',
    'Jonkheer':'Royalty',
    'Don'     :'Royalty',
    'Sir'     :'Royalty',
    'Dr':'Officer',
    'Rev':'Officer',
    'the Countess':'Royalty',
    'Dona':'Royalty',
    'Mme':'Mrs',
    'Mlle':'Miss',
    'Ms':'Mrs',
    'Mr':'Mr',
    'Mrs':'Mrs',
    'Miss':'Miss',
    'Master':'Master',
    'Lady':'Royalty'
    
}

In [20]:
### 将title 转成one-hot 变量的形式
Title_DF['Title'] = Title_DF['Title'].map(map_dict)
Title_DF = pd.get_dummies(Title_DF['Title'])
Title_DF.head(2)

,Master,Miss,Mr,Mrs,Officer,Royalty
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
### 将Title_tmp 拼接到数据样本full中
full = pd.concat([full,Title_DF],axis =1 )
full.drop('Name',axis=1,inplace = 'True') # 删掉‘Name’这一列
full.head(2)

,Age,Cabin,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,...,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Master,Miss,Mr,Mrs,Officer,Royalty
0,22.0,UNK,7.2500,0,1,1,1,0.0,A/5 21171,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,C85,71.2833,0,2,0,1,1.0,PC 17599,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### 2.5 将旅客所在的客舱所在的'Cabin' 利用one-hot 向量来表示

In [22]:
full['Cabin'].head()

0     UNK
1     C85
2     UNK
3    C123
4     UNK
Name: Cabin, dtype: object

In [23]:
### 抽取第一个字母作为分类区别并通过one-hot编码表示
full['Cabin'] = full['Cabin'].map(lambda x:x[0])
Cabin_DF = pd.DataFrame()
Cabin_DF = pd.get_dummies(full['Cabin'],prefix = 'Cabin')
Cabin_DF.head()

,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
# 将Cabin_tmp 添加到full 列中并去掉原始的'Cabin'列
full = pd.concat([full,Cabin_DF],axis = 1)
full.drop('Cabin',axis = 1,inplace = 'True') # delete 掉原始的‘Cabin’列
full.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,...,Royalty,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,22.0,7.2500,0,1,1,1,0.0,A/5 21171,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,38.0,71.2833,0,2,0,1,1.0,PC 17599,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26.0,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,35.0,53.1000,0,4,0,1,1.0,113803,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35.0,8.0500,0,5,1,0,0.0,373450,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 2.6 整理家庭列别的分类 - 按照家庭的人数来将大小分 小-中-大 三种分类结果

In [25]:
FamilyDF  = pd.DataFrame()
FamilyDF['family_size'] = full['Parch'] + full['SibSp'] +1 
# 哪找家庭人 分 min/medim/max 三类
FamilyDF['family_min'] = FamilyDF['family_size'].map(lambda x: 1 if x<2 else 0)
FamilyDF['family_medim'] = FamilyDF['family_size'].map(lambda x: 1 if 2<=x<=4 else 0)
FamilyDF['family_large'] = FamilyDF['family_size'].map(lambda x: 1 if x>4 else 0)
FamilyDF.head()

,family_size,family_min,family_medim,family_large
0,2,0,1,0
1,2,0,1,0
2,1,1,0,0
3,2,0,1,0
4,1,1,0,0


In [26]:
### FamilyDF 加full中
full = pd.concat([full,FamilyDF],axis = 1)
#full.head()

### 2.7 将年龄按照大小分类

In [27]:
### 2.7 将年龄按照大小分类
AgeDF = pd.DataFrame()
AgeDF['Child']    = full['Age'].map(lambda x:1 if 0<x<6 else 0)
AgeDF['Teenager'] = full['Age'].map(lambda x:1 if 6<=x<18 else 0)
AgeDF['Youth']    = full['Age'].map(lambda x:1 if 18<=x<40 else 0)
AgeDF['Mid_Aged'] = full['Age'].map(lambda x:1 if 40<=x<=60 else 0)
AgeDF['Old_Aged'] = full['Age'].map(lambda x:1 if 60<x else 0)

In [28]:
full.shape,
full.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Ticket,Embarked_C,Embarked_Q,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,family_size,family_min,family_medim,family_large
0,22.0,7.2500,0,1,1,1,0.0,A/5 21171,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,0,1,0
1,38.0,71.2833,0,2,0,1,1.0,PC 17599,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0
2,26.0,7.9250,0,3,0,0,1.0,STON/O2. 3101282,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0,0
3,35.0,53.1000,0,4,0,1,1.0,113803,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0,1,0
4,35.0,8.0500,0,5,1,0,0.0,373450,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0,0


### 2.8 计算各个特征更便签的相关特性 - 相关矩阵

In [29]:
### 2.8 计算各个特征更便签的相关特性 - 相关矩阵
Corr_Matrix = full.corr()
Corr_Matrix.head(2)

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,family_size,family_min,family_medim,family_large
Age,1.000000,0.171521,-0.130872,0.025731,0.057397,-0.190747,-0.070323,0.076179,-0.012718,-0.059153,...,0.132886,0.106600,-0.072644,-0.085977,0.032461,-0.271918,-0.196996,0.116675,-0.038189,-0.161210
Fare,0.171521,1.000000,0.221522,0.031416,-0.185484,0.160224,0.257307,0.286241,-0.130054,-0.169894,...,0.072737,0.073949,-0.037567,-0.022857,0.001179,-0.507197,0.226465,-0.274826,0.197281,0.170853


### 2.9 查找相关性比较高的特征

In [30]:
Corr_Matrix["Survived"].sort_values(ascending = False)

Survived        1.000000
Mrs             0.344935
Miss            0.332795
Pclass_1        0.285904
family_medim    0.279855
Fare            0.257307
Cabin_B         0.175095
Embarked_C      0.168240
Cabin_D         0.150716
Cabin_E         0.145321
Cabin_C         0.114652
Pclass_2        0.093349
Master          0.085221
Parch           0.081629
Cabin_F         0.057935
Royalty         0.033391
Cabin_A         0.022287
family_size     0.016639
Cabin_G         0.016040
Embarked_Q      0.003650
PassengerId    -0.005007
Cabin_T        -0.026456
Officer        -0.031316
SibSp          -0.035322
Age            -0.070323
family_large   -0.125147
Embarked_S     -0.149683
family_min     -0.203367
Cabin_U        -0.316912
Pclass_3       -0.322308
Sex            -0.543351
Mr             -0.549199
Name: Survived, dtype: float64

### 2.10通过相似度计算来确定的正面特征是

In [31]:
### 确定的正面特征是
#  Title_DF
#  Pclass_DF，
#  FamilyDF,full['Fare']
#  full['Sex'],Cabin_DF,Embarked_DF

### 3 确定最终的特征

In [32]:
full_X = pd.concat([Title_DF, #头衔
                 Pclass_DF,   #客舱等级
                  FamilyDF,   # 家庭大小
                  full['Fare'],# 票的价钱
                  full['Sex'], # 旅客的性别# Cabin_DF客舱号码
                  Embarked_DF],axis =1 ) #登船的港口

### 4.样本分割及建立模型

In [33]:
LENS_TRAIN_DATA = 891
X = full_X.loc[0:LENS_TRAIN_DATA -1,:]
y = full.loc[0:LENS_TRAIN_DATA -1,'Survived']

#print('X size is :',X.shape[0])
#print('y size is :',y.shape[0])

# 需要被测试的数据集合大小Text_X
Text_X = full_X.loc[LENS_TRAIN_DATA:,:]
#print("Text_X size is ",Text_X.shape[0])

In [34]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [35]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
print("逻辑回归基本模型的测试准确率是%0.2f"%(model.score(X_test,y_test)*100),'%')

逻辑回归基本模型的测试准确率是87.71 %


### 4.通过网格搜索交叉验证来调剂参数

In [36]:
from sklearn.model_selection import GridSearchCV
param_grid  = {
    'C':[2,1,3],
    'max_iter':[1,2,5,10,50],
    'solver':['liblinear'],
    'penalty':['l2','l1'],
    'tol':[1e-3,1e-4,1e-2]
}
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)
models = GridSearchCV(model,param_grid ,n_jobs =-1)
models.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_iter': [1, 2, 5, 10, 50], 'penalty': ['l2', 'l1'], 'C': [2, 1, 3], 'solver': ['liblinear'], 'tol': [0.001, 0.0001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [37]:
models.best_params_ ,models.best_score_,models.best_estimator_

({'C': 1,
  'max_iter': 2,
  'penalty': 'l1',
  'solver': 'liblinear',
  'tol': 0.0001},
 0.8160112359550562,
 LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=2, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False))

In [38]:
best_estimator = models.best_estimator_

In [39]:
print("网格搜索后逻辑回归模型的测试准确率是%0.2f"%(best_estimator.score(X_test,y_test)*100),'%')

网格搜索后逻辑回归模型的测试准确率是85.47 %


In [46]:
from sklearn import cross_validation
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(best_estimator,X,y,cv = kf)
print("最优逻辑回归模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

最优逻辑回归模型交叉验证后的预测准确率是82.83 %


#### ================分割线======================##

### 5. 引入SVM模型

In [40]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [41]:
from sklearn.model_selection import GridSearchCV
param_grid  = {
    'C':[2,1,3,4],
    'kernel':('linear','poly','rbf','sigmoid'),
    'degree':[1,2,3,4],
    'coef0':[0,1,2],
    'max_iter':[1500,2000,3000],
    'decision_function_shape':['ovo','ovr'],
    'probability':[True]
}
model_SVM = GridSearchCV(svc_clf,param_grid ,n_jobs =-1)
model_SVM.fit(X_train,y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_iter': [1500, 2000, 3000], 'kernel': ('linear', 'poly', 'rbf', 'sigmoid'), 'coef0': [0, 1, 2], 'probability': [True], 'degree': [1, 2, 3, 4], 'C': [2, 1, 3, 4], 'decision_function_shape': ['ovo', 'ovr']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [42]:
model_SVM.best_score_,model_SVM.best_params_

(0.8202247191011236,
 {'C': 1,
  'coef0': 1,
  'decision_function_shape': 'ovo',
  'degree': 1,
  'kernel': 'poly',
  'max_iter': 3000,
  'probability': True})

In [43]:
best_model_ = model_SVM.best_estimator_

In [44]:
print("网格搜索后SVM模型的测试准确率是%0.2f"%(best_model_.score(X_test,y_test)*100),'%')

网格搜索后SVM模型的测试准确率是87.71 %


In [45]:
from sklearn import cross_validation
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(best_model_,X,y,cv = kf)
print("最优随机森林模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


最优随机森林模型交叉验证后的预测准确率是82.94 %


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


#### ================分割线======================##

### 6. 引入随机森林模型

In [47]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [49]:
param_grid = {
    'n_estimators':[100,150,200],
    'criterion':['gini'],
    'max_depth':[2,5],
    'min_samples_split':[2,3],
    'min_samples_leaf':[1],
    'max_leaf_nodes':[7,12,15,20],
    'bootstrap':[True]
}
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)
clf_RF    = RandomForestClassifier()
clf_RF_cv = GridSearchCV(clf_RF,param_grid,n_jobs = -1)
clf_RF_cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini'], 'bootstrap': [True], 'min_samples_leaf': [1], 'n_estimators': [100, 150, 200], 'max_depth': [2, 5], 'max_leaf_nodes': [7, 12, 15, 20], 'min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [50]:
clf_RF_cv.best_score_,clf_RF_cv.best_params_

(0.8286516853932584,
 {'bootstrap': True,
  'criterion': 'gini',
  'max_depth': 5,
  'max_leaf_nodes': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 150})

In [51]:
best_RF = clf_RF_cv.best_estimator_

In [52]:
print("网格搜索后随机森林的测试准确率是%0.2f"%(best_RF.score(X_test,y_test)*100),'%')

网格搜索后随机森林的测试准确率是86.59 %


In [53]:
from sklearn import cross_validation
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(best_RF,X,y,cv = kf)
print("最优随机森林模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

最优随机森林模型交叉验证后的预测准确率是82.49 %


#### ================分割线======================##

### 7.引入决策树模型

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion':['gini','entropy'],
    'splitter':['best'],
    'max_depth':[None,1,2,3],
    'min_samples_split':[2,3,4],
    'min_samples_leaf':[1,2],
    'max_leaf_nodes':[3,5,7]
}
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)

DT = DecisionTreeClassifier()
DT_CV = GridSearchCV(DT,param_grid,n_jobs = -1)
DT_CV.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2], 'splitter': ['best'], 'max_depth': [None, 1, 2, 3], 'max_leaf_nodes': [3, 5, 7], 'min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [55]:
DT_CV.best_score_,DT_CV.best_params_

(0.8146067415730337,
 {'criterion': 'gini',
  'max_depth': None,
  'max_leaf_nodes': 7,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'splitter': 'best'})

In [56]:
best_DT = DT_CV.best_estimator_
print("网格搜索后决策树模型测试准确率是%0.2f"%(best_DT.score(X_test,y_test)*100),'%')

网格搜索后决策树模型测试准确率是85.47 %


In [57]:
from sklearn import cross_validation
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(best_DT,X,y,cv = kf)
print("最优决策树模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

最优决策树模型交叉验证后的预测准确率是81.93 %


#### ================分割线======================##

### 8 引入集成学习的思想

In [58]:
### 8.1 Hard voting classifier 
from sklearn.ensemble import VotingClassifier
voting_classifier = VotingClassifier(
estimators = [
        ('logistic',best_estimator),
        ('svm',best_model_),
        ('randomforest',best_RF),
        ('DecisionTree',best_DT)], voting = 'hard'
)

In [59]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)
ensembels_hard = voting_classifier.fit(X_train,y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [60]:
print("Hard集成学习的准确率是%0.2f"%(ensembels_hard.score(X_test,y_test)*100),'%')

Hard集成学习的准确率是86.59 %


In [61]:
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(ensembels_hard,X,y,cv = kf)
print("最优集成学习模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


最优集成学习模型交叉验证后的预测准确率是82.83 %


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [62]:
### 8.2 Soft voting classifier 
from sklearn.ensemble import VotingClassifier
voting_classifier = VotingClassifier(
estimators = [
        ('logistic',best_estimator),
        ('svm',best_model_),
        ('randomforest',best_RF),
        ('DecisionTree',best_DT)], voting = 'soft'
)

In [63]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)
ensembels_soft = voting_classifier.fit(X_train,y_train)

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [64]:
print("Soft集成学习的准确率是%0.2f"%(ensembels_soft.score(X_test,y_test)*100),'%')

Soft集成学习的准确率是87.15 %


In [65]:
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(ensembels_soft,X,y,cv = kf)
print("最优集成学习Soft模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


最优集成学习Soft模型交叉验证后的预测准确率是82.71 %


C:\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=3000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [67]:
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(DecisionTreeClassifier(),
                           n_estimators=500, max_samples=100,
                           bootstrap=True)
bagging_clf.fit(X_train, y_train)
bagging_clf.score(X_test, y_test)*100

86.033519553072622

In [68]:
param_grid = {
    'n_estimators':[300,500,700],
    'max_samples':[100,300],
    'bootstrap':[True]
}
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =180)

Bagging_clf = BaggingClassifier()
Bagging_clf = GridSearchCV(Bagging_clf,param_grid,n_jobs = -1)
Bagging_clf.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_samples': [100, 300], 'n_estimators': [300, 500, 700]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [70]:
Bagging_clf.best_score_,Bagging_clf.best_params_

(0.8216292134831461,
 {'bootstrap': True, 'max_samples': 100, 'n_estimators': 700})

In [71]:
best_bag_clf = Bagging_clf.best_estimator_

In [72]:
kf     = cross_validation.KFold(LENS_TRAIN_DATA, n_folds=5, random_state=180)
scores = cross_validation.cross_val_score(best_bag_clf,X,y,cv = kf)
print("最优bagging_clf模型交叉验证后的预测准确率是%0.2f"%(scores.mean()*100),'%')

最优bagging_clf模型交叉验证后的预测准确率是82.72 %


### X. 对需要预测的数据来计算预测的结果 - 80%

In [84]:
### 下面获得准确率大约是80%
pre_y = best_model.predict(Text_X)
pred_y = pre_y.astype(int)
PassengerId = full.loc[LENS_TRAIN_DATA:,'PassengerId']
pred_DF = pd.DataFrame({'PassengerId':PassengerId,
                        'Survived':pred_y})
pred_DF.to_csv("taitannike_predict.csv",index = False)
print(pred_DF.shape)
pred_DF.head()

(418, 2)


,PassengerId,Survived
891,892,0
892,893,1
893,894,0
894,895,0
895,896,1
